# Rules of persona
+ Each sentence must contain between 4 and 20 words or punctuation marks.
+ It contains either the word I or my.
+ At least one verb, and (iv) at least one noun, pronoun or adjective.

# Example Dialogue with persona
- Persona: [“I like sport”, “I work a lot”]
- Context: “I love running.”
- Response: “Me too! But only on weekends.”

{
    
    "dialog": [["没有 钱   万万 不行 ！ ~"], ["现实 就是 如此"]], 
    
    "profile": [{"tag": ["漫画;旅遊;星座"], "loc": "广东 广州", "gender": "male"}, {"tag": [""], "loc": "", "gender": ""}], 
    
    "uid": [0, 1]

}

{
    
    "dialog": [["For what it's worth, I don't have a problem with it."], ["My apologies.  I did not have any problems with it, but I will be more careful in the future."]], 
    
    "profile": [{"tag": [], "loc": "", "gender": ""}, {"tag": ["i did not have any problems with it, but i will be more careful in the future."], 
    
    "loc": "", "gender": ""}], 
    
    "uid": [0, 1]

}

# test_data Example
{"uid": [0, 1, 2], "dialog": [["剧烈运动 是 吧"], ["各种 剧烈运动"], ["... 姐 最近 有点 寂寞 过头 了 ..."]], "responder_profile": {"loc": "海南", "gender": "female", "tag": "美食;宅;80后"}, "profile": [{"loc": "天津 滨海新区", "gender": "male", "tag": ""}, {"loc": "海南", "gender": "female", "tag": "美食;宅;80后"}, {"loc": "安徽 合肥", "gender": "male", "tag": "游戏动漫;双子座;宅;音乐;90后;WOW台服众"}], "golden_response": ["可不是 ， 我 又 不 像 你 ， 有 女神 。"]}

# Constant Value

In [334]:
NPARTITIONS = 1000
PATH = "./reddit_data/*/*.json"
SCHEDULER = "threads"

# Imports

In [335]:
import pandas as pd 
import json
import bz2
from tqdm import tqdm
import glob
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import spacy
import os
import redditcleaner
import neuralcoref

In [336]:
tqdm.pandas()
ProgressBar().register()
nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)

#doc1 = nlp('My sister has a dog. She loves him.')
#print(doc1._.coref_resolved)

In [337]:
if(not os.path.exists("./outputs")):
    os.makedirs("./outputs/")


In [338]:
version = len([f for f in os.listdir("./outputs") if "persona" in f])
version

3

# reddit_data下の全てのjsonファイルを読み込む

In [339]:
list_bz2_file = glob.glob(PATH)
list_reddit_conversation = []
list_bz2_file

['./reddit_data\\2007\\RC_2007-10.json']

In [340]:
print("----------read input json files----------")
for i in tqdm(range(0,len(list_bz2_file))):
    with open(list_bz2_file[i], mode="r", encoding="shift-jis") as f:
        for line in f.readlines():
            dic=json.loads(line)
            list_reddit_conversation.append(dic)

  0%|          | 0/1 [00:00<?, ?it/s]

----------read input json files----------


100%|██████████| 1/1 [00:00<00:00,  1.37it/s]


In [341]:
df_reddit_conversation = pd.DataFrame(list_reddit_conversation)
df_reddit_conversation = df_reddit_conversation[df_reddit_conversation["author"]!="[deleted]"]
df_reddit_conversation["body"] = df_reddit_conversation["body"].progress_map(lambda x:redditcleaner.clean(str(x)))
df_reddit_conversation["body"] = df_reddit_conversation["body"].replace(["&lt","&gt","&amp"],["","",""])
df_reddit_conversation["body"] = df_reddit_conversation["body"].replace(["\\n+","\\r+","////","”","’"],["","","","",""], regex=True)
df_reddit_conversation.to_csv(f"./outputs/AllConversation{version}.csv")
df_reddit_conversation.head(5)

100%|██████████| 9993/9993 [00:00<00:00, 35808.60it/s]


,parent_id,created_utc,ups,controversiality,distinguished,subreddit_id,id,downs,archived,link_id,...,author,score_hidden,body,gilded,author_flair_text,subreddit,edited,author_flair_css_class,name,retrieved_on
0,t3_5yba3,1192450635,1,0,None,t5_6,c0299an,0,True,t3_5yba3,...,bostich,False,test,0,None,reddit.com,False,None,t1_c0299an,1427426409
1,t3_5yba3,1192450639,2,0,None,t5_6,c0299ao,0,True,t3_5yba3,...,igiveyoumylife,False,"much smoother. Im just glad reddit is back, re...",0,None,reddit.com,False,None,t1_c0299ao,1427426409
2,t1_c02999p,1192450643,0,0,None,t5_6,c0299ap,0,True,t3_5yba3,...,Arve,False,"Can we please deprecate the word ""Ajax"" now? (...",0,None,reddit.com,False,None,t1_c0299ap,1427426409
4,t1_c0299ah,1192450646,3,0,None,t5_6,c0299ar,0,True,t3_5yba3,...,gigaquack,False,"Oh, I see. Fancy schmancy ""submitting....""",0,None,reddit.com,False,None,t1_c0299ar,1427426409
5,t3_5yba3,1192450656,1,0,None,t5_6,c0299as,0,True,t3_5yba3,...,Percept,False,testing ...,0,None,reddit.com,False,None,t1_c0299as,1427426409


In [353]:
df_reddit_conversation[df_reddit_conversation["body"].str.contains("”")]

KeyError: "None of [Float64Index([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n              ...\n              nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n             dtype='float64', length=4159)] are in the [columns]"

# 会話ペアの作成

In [342]:
df_reddit_conversation["removed_prefix_parent_id"] = df_reddit_conversation["parent_id"].str.replace("t\d_","")
df_reddit_conversation["parent_body"] = df_reddit_conversation[df_reddit_conversation["removed_prefix_parent_id"]==df_reddit_conversation["id"]]["body"]
df_reddit_conversation["body"] = df_reddit_conversation["body"].str.replace('\"','’')
df_reddit_conversation["parent_body"] = df_reddit_conversation["parent_body"].str.replace('\"','’')
df_reddit_conversation = pd.merge(df_reddit_conversation,df_reddit_conversation[["id","body"]].rename(columns={"id":"parent_id","body":"parent_body"}),left_on="removed_prefix_parent_id",right_on="parent_id").drop(columns=["parent_body_x","parent_id_y"]).rename(columns={"parent_body_y":"parent_body"})
df_reddit_conversation = df_reddit_conversation.dropna(subset=["parent_body"]).sort_values(["author"]).reset_index(drop=True)
df_reddit_conversation["original_body"] = df_reddit_conversation["body"]
df_reddit_conversation["original_parent_body"] = df_reddit_conversation["parent_body"]
df_reddit_conversation = df_reddit_conversation[["body","parent_body","original_body","original_parent_body","ups","author"]]
df_reddit_conversation

<ipython-input-342-15e556bf2943>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_reddit_conversation["removed_prefix_parent_id"] = df_reddit_conversation["parent_id"].str.replace("t\d_","")


,body,parent_body,original_body,original_parent_body,ups,author
0,"I'd like to agree with you, but did you see wh...",Wrong. You'd better have an idea of why the gu...,"I'd like to agree with you, but did you see wh...",Wrong. You'd better have an idea of why the gu...,16,48klocs
1,Sitting at 22. Now you can begin shaving.,"Sorry, default 25'er front page only. P",Sitting at 22. Now you can begin shaving.,"Sorry, default 25'er front page only. P",7,54gy6dm
2,that was subtle... thanks.,genetically identical fish are sending out tw...,that was subtle... thanks.,genetically identical fish are sending out tw...,4,54gy6dm
3,I don't have to prove anything. I can ask any ...,"Again, prove that. If they think they are neve...",I don't have to prove anything. I can ask any ...,"Again, prove that. If they think they are neve...",1,60MinuteMan
4,AHAHHAHAHAHAHAH YOU TOTALLY SUCK. http://www.n...,"Thanks, repug.",AHAHHAHAHAHAHAH YOU TOTALLY SUCK. http://www.n...,"Thanks, repug.",0,60MinuteMan
...,...,...,...,...,...,...
4154,"Nope. Months of development, a day of downtime...",Is anything else different? Surely there's mor...,"Nope. Months of development, a day of downtime...",Is anything else different? Surely there's mor...,4,zoomzoom83
4155,Thanks for that- I always assumed the kinetic ...,"Not that it changes your point, but I think yo...",Thanks for that- I always assumed the kinetic ...,"Not that it changes your point, but I think yo...",4,zoomzoom83
4156,No,Can we please deprecate the word ’Ajax’ now? (...,No,Can we please deprecate the word ’Ajax’ now? (...,2,zoomzoom83
4157,http://www.fff.org/comment/com0311c.asp maybe ...,Wow What a shocker After Saddam attempted to a...,http://www.fff.org/comment/com0311c.asp maybe ...,Wow What a shocker After Saddam attempted to a...,1,zorno


In [343]:
def CreatePersona(body: str):
    doc = nlp(body.lower())
    # 文ごとに分割
    persona = [str(sentence) for sentence in doc.sents if IsPersona(str(sentence))]
    return persona

In [344]:
def IsPersona(sentence: str):
    # 以下の3つの条件を満たすものをペルソナとする
    # 1.文の単語数が4-20の間
    # 2.I か my　が含まれている
    # 3.少なくとも1つの動詞と，名詞，代名詞，形容詞のいずれかが含まれている
    words = [str(word) for word in nlp(sentence.strip())]
    poses = [token.pos_ for token in nlp(sentence.strip())]
    return (
        (4 <= len(words) <= 20)&
        (not set(["i","my"]).isdisjoint(set(words)))&
        (("VERB" in poses)&(not set(["NOUN", "ADJ", "PROPN"]).isdisjoint(set(poses))))
    )

In [345]:
def create_json(row):
    return {
        "dialog":row["dialog"],
        "profile":[
            {"tag":row["persona"],
            "loc":"",
            "gender":""},
            {"tag":row["parent_persona"],
            "loc":"",
            "gender":""}
        ],
        "uid":[0,1]
    }

# ペルソナの作成

In [346]:
print("----------create conversation pair ----------")
ddf_reddit_conversation = dd.from_pandas(data=df_reddit_conversation, npartitions=NPARTITIONS)
ddf_reddit_conversation["persona"] = ddf_reddit_conversation["original_body"].map(CreatePersona)
ddf_reddit_conversation["parent_persona"] = ddf_reddit_conversation["original_parent_body"].map(CreatePersona)
ddf_reddit_conversation.query("persona.notnull() | parent_persona.notnull()")
#ddf_reddit_conversation["body"] = ddf_reddit_conversation["body"].map(lambda sentence:nlp(sentence)._.coref_resolved)
#ddf_reddit_conversation["parent_body"] = ddf_reddit_conversation["parent_body"].map(lambda sentence:nlp(sentence)._.coref_resolved)
df_reddit_conversation = ddf_reddit_conversation.compute(scheduler=SCHEDULER)

----------create conversation pair ----------
[########################################] | 100% Completed | 14min 17.0s
[########################################] | 100% Completed | 14min 17.1s
[########################################] | 100% Completed | 14min 17.2s
[########################################] | 100% Completed | 14min 17.2s
[########################################] | 100% Completed | 14min 17.2s
[########################################] | 100% Completed | 14min 17.3s
[########################################] | 100% Completed | 14min 17.4s
[########################################] | 100% Completed | 14min 17.4s
[########################################] | 100% Completed | 14min 17.5s
[########################################] | 100% Completed | 14min 17.5s
[########################################] | 100% Completed | 14min 17.6s
[########################################] | 100% Completed | 14min 17.6s
[########################################] | 100% Completed | 14mi

In [347]:
print("--------- create list ----------")
df_reddit_conversation["body"] = df_reddit_conversation["body"].progress_map(lambda x: [x] )
df_reddit_conversation["parent_body"] = df_reddit_conversation["parent_body"].progress_map(lambda x: [x] )
df_reddit_conversation["dialog"] = [list(x) for x in zip(df_reddit_conversation["body"].tolist(),df_reddit_conversation["parent_body"].tolist())]
df_reddit_conversation

100%|██████████| 4159/4159 [00:00<00:00, 1039391.67it/s]

--------- create list ----------


,body,parent_body,original_body,original_parent_body,ups,author,persona,parent_persona,dialog
0,"[I'd like to agree with you, but did you see w...",[Wrong. You'd better have an idea of why the g...,"I'd like to agree with you, but did you see wh...",Wrong. You'd better have an idea of why the gu...,16,48klocs,[],[],"[[I'd like to agree with you, but did you see ..."
1,[Sitting at 22. Now you can begin shaving.],"[Sorry, default 25'er front page only. P]",Sitting at 22. Now you can begin shaving.,"Sorry, default 25'er front page only. P",7,54gy6dm,[],[],"[[Sitting at 22. Now you can begin shaving.], ..."
2,[that was subtle... thanks.],[ genetically identical fish are sending out t...,that was subtle... thanks.,genetically identical fish are sending out tw...,4,54gy6dm,[],[],"[[that was subtle... thanks.], [ genetically i..."
3,[I don't have to prove anything. I can ask any...,"[Again, prove that. If they think they are nev...",I don't have to prove anything. I can ask any ...,"Again, prove that. If they think they are neve...",1,60MinuteMan,[],[i think you're generalizing quite a bit witho...,[[I don't have to prove anything. I can ask an...
4,[AHAHHAHAHAHAHAH YOU TOTALLY SUCK. http://www....,"[Thanks, repug.]",AHAHHAHAHAHAHAH YOU TOTALLY SUCK. http://www.n...,"Thanks, repug.",0,60MinuteMan,[],[],[[AHAHHAHAHAHAHAH YOU TOTALLY SUCK. http://www...
...,...,...,...,...,...,...,...,...,...
4154,"[Nope. Months of development, a day of downtim...",[Is anything else different? Surely there's mo...,"Nope. Months of development, a day of downtime...",Is anything else different? Surely there's mor...,4,zoomzoom83,"[(in reality, i think it's a complete rewrite ...","[edit: i guess that sounded kinda snotty., i r...","[[Nope. Months of development, a day of downti..."
4155,[Thanks for that- I always assumed the kinetic...,"[Not that it changes your point, but I think y...",Thanks for that- I always assumed the kinetic ...,"Not that it changes your point, but I think yo...",4,zoomzoom83,[],"[not that it changes your point, but i think y...",[[Thanks for that- I always assumed the kineti...
4156,[No],[Can we please deprecate the word ’Ajax’ now? ...,No,Can we please deprecate the word ’Ajax’ now? (...,2,zoomzoom83,[],[],"[[No], [Can we please deprecate the word ’Ajax..."
4157,[http://www.fff.org/comment/com0311c.asp maybe...,[Wow What a shocker After Saddam attempted to ...,http://www.fff.org/comment/com0311c.asp maybe ...,Wow What a shocker After Saddam attempted to a...,1,zorno,[],[],[[http://www.fff.org/comment/com0311c.asp mayb...


# Json形式の作成

In [348]:
df_reddit_conversation["json"] = df_reddit_conversation.progress_apply(create_json, axis=1)
df_reddit_conversation

100%|██████████| 4159/4159 [00:00<00:00, 71691.17it/s]


,body,parent_body,original_body,original_parent_body,ups,author,persona,parent_persona,dialog,json
0,"[I'd like to agree with you, but did you see w...",[Wrong. You'd better have an idea of why the g...,"I'd like to agree with you, but did you see wh...",Wrong. You'd better have an idea of why the gu...,16,48klocs,[],[],"[[I'd like to agree with you, but did you see ...","{'dialog': [['I'd like to agree with you, but ..."
1,[Sitting at 22. Now you can begin shaving.],"[Sorry, default 25'er front page only. P]",Sitting at 22. Now you can begin shaving.,"Sorry, default 25'er front page only. P",7,54gy6dm,[],[],"[[Sitting at 22. Now you can begin shaving.], ...",{'dialog': [['Sitting at 22. Now you can begin...
2,[that was subtle... thanks.],[ genetically identical fish are sending out t...,that was subtle... thanks.,genetically identical fish are sending out tw...,4,54gy6dm,[],[],"[[that was subtle... thanks.], [ genetically i...","{'dialog': [['that was subtle... thanks.'], ['..."
3,[I don't have to prove anything. I can ask any...,"[Again, prove that. If they think they are nev...",I don't have to prove anything. I can ask any ...,"Again, prove that. If they think they are neve...",1,60MinuteMan,[],[i think you're generalizing quite a bit witho...,[[I don't have to prove anything. I can ask an...,{'dialog': [['I don't have to prove anything. ...
4,[AHAHHAHAHAHAHAH YOU TOTALLY SUCK. http://www....,"[Thanks, repug.]",AHAHHAHAHAHAHAH YOU TOTALLY SUCK. http://www.n...,"Thanks, repug.",0,60MinuteMan,[],[],[[AHAHHAHAHAHAHAH YOU TOTALLY SUCK. http://www...,{'dialog': [['AHAHHAHAHAHAHAH YOU TOTALLY SUCK...
...,...,...,...,...,...,...,...,...,...,...
4154,"[Nope. Months of development, a day of downtim...",[Is anything else different? Surely there's mo...,"Nope. Months of development, a day of downtime...",Is anything else different? Surely there's mor...,4,zoomzoom83,"[(in reality, i think it's a complete rewrite ...","[edit: i guess that sounded kinda snotty., i r...","[[Nope. Months of development, a day of downti...","{'dialog': [['Nope. Months of development, a d..."
4155,[Thanks for that- I always assumed the kinetic...,"[Not that it changes your point, but I think y...",Thanks for that- I always assumed the kinetic ...,"Not that it changes your point, but I think yo...",4,zoomzoom83,[],"[not that it changes your point, but i think y...",[[Thanks for that- I always assumed the kineti...,{'dialog': [['Thanks for that- I always assume...
4156,[No],[Can we please deprecate the word ’Ajax’ now? ...,No,Can we please deprecate the word ’Ajax’ now? (...,2,zoomzoom83,[],[],"[[No], [Can we please deprecate the word ’Ajax...","{'dialog': [['No'], ['Can we please deprecate ..."
4157,[http://www.fff.org/comment/com0311c.asp maybe...,[Wow What a shocker After Saddam attempted to ...,http://www.fff.org/comment/com0311c.asp maybe ...,Wow What a shocker After Saddam attempted to a...,1,zorno,[],[],[[http://www.fff.org/comment/com0311c.asp mayb...,{'dialog': [['http://www.fff.org/comment/com03...


# Outputs

In [349]:
df_reddit_conversation.to_csv(f"./outputs/persona{version}.csv")

In [350]:
list_json = df_reddit_conversation["json"].tolist()
with open(f"./outputs/created_dialogues{version}.json", "wt", encoding="utf-8") as file:
    for dic in list_json:
        file.write(str(json.dumps(dic))+"\n")

In [351]:
import subprocess
subprocess.run(['jupyter', 'nbconvert', '--to', 'script', '*.ipynb'])

CompletedProcess(args=['jupyter', 'nbconvert', '--to', 'script', '*.ipynb'], returncode=0)